In [23]:
import pandas as pd
import numpy as np

df = pd.read_csv('CRL_1_raw.csv').drop('Unnamed: 0', axis=1)
#df.gameMode_name.unique()

In [24]:
#only select proper game modes
df_clean = df[(df['gameMode_name']=='Ladder') | (df['gameMode_name']=='Friendly') | (df['gameMode_name']=='Overtime_Friendly') | (df['gameMode_name']=='Tournament') | (df['gameMode_name']=='Ladder_CrownRush') | (df['gameMode_name']=='Ladder_GoldRush') | (df['gameMode_name']=='CW_Battle_1v1') | (df['gameMode_name']=='CW_Duel_1v1')]

#only select decks, crowns are irrelevant
df_1 = df_clean[[
    'team_0_cards_0_name', 
    'team_0_cards_1_name', 
    'team_0_cards_2_name', 
    'team_0_cards_3_name', 
    'team_0_cards_4_name', 
    'team_0_cards_5_name', 
    'team_0_cards_6_name', 
    'team_0_cards_7_name']]
df_1.columns = list(range(8))

df_2 = df_clean[[
    'opponent_0_cards_0_name', 
    'opponent_0_cards_1_name',
    'opponent_0_cards_2_name', 
    'opponent_0_cards_3_name',
    'opponent_0_cards_4_name', 
    'opponent_0_cards_5_name',
    'opponent_0_cards_6_name', 
    'opponent_0_cards_7_name']]
df_2.columns = list(range(8))

#put everything together
df_clean = pd.concat([df_1, df_2], axis=0)
df_clean.to_csv('CRL_2_matches.csv')
df_clean

,0,1,2,3,4,5,6,7
0,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
1,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Ice Golem,Magic Archer,Barbarian Barrel,Tombstone
2,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
3,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
4,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
...,...,...,...,...,...,...,...,...
16563,Elite Barbarians,Elixir Collector,Bandit,Heal Spirit,Archer Queen,Royal Ghost,Barbarian Barrel,Three Musketeers
16564,Electro Giant,Bowler,Musketeer,Goblin Cage,Skeletons,Barbarian Barrel,Tornado,Lightning
16565,Ice Golem,Hog Rider,Fireball,Ice Spirit,Musketeer,The Log,Cannon,Skeletons
16566,P.E.K.K.A,Electro Wizard,Bandit,Battle Ram,Dark Prince,Fireball,Zap,Magic Archer


## index-based encoding

In [25]:
#index-based encoding
from sklearn import preprocessing
codex = preprocessing.LabelEncoder()
codex = codex.fit(pd.unique(df_clean.values.ravel('K'))) #all unique values in a dataset
#codex.transform(['Battle Healer'])
#codex.inverse_transform([10])

df_index = df_clean.apply(lambda x : codex.transform(x))
df_index

,0,1,2,3,4,5,6,7
0,79,56,61,30,5,62,6,53
1,79,56,61,30,53,62,6,97
2,79,56,61,30,5,62,6,53
3,79,56,61,30,5,62,6,53
4,79,56,61,30,5,62,6,53
...,...,...,...,...,...,...,...,...
16563,25,26,5,50,0,83,6,96
16564,22,14,72,40,91,6,98,60
16565,53,51,30,54,72,95,15,91
16566,74,24,5,11,18,30,104,62


## one_hot encoding

In [26]:
#one_hot encoding
row_list = list()
#len(pd.unique(df_index.values.ravel('K')))
for row in range(df_index.shape[0]):
    a = np.array(df_index.iloc[row])
    b = np.zeros((a.size, 106))
    b[np.arange(a.size),a] = 1
    row_list.append(np.sum(b, axis=0))

df_onehot = pd.DataFrame(row_list) #.astype(int)
df_onehot.columns = [codex.inverse_transform([x])[0] for x in list(df_onehot.columns)]
df_onehot.to_csv('CRL_3_encoded.csv')
df_onehot

,Archer Queen,Archers,Arrows,Baby Dragon,Balloon,Bandit,Barbarian Barrel,Barbarian Hut,Barbarians,Bats,...,Three Musketeers,Tombstone,Tornado,Valkyrie,Wall Breakers,Witch,Wizard,X-Bow,Zap,Zappies
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23671,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23672,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23674,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
